In [4]:
import pandas as pd
import numpy as np
import scipy.sparse
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('/content/preprocessed_reviews.csv')

# Get the number of reviews for each product
product_review_count = df['app_name'].value_counts()


print(product_review_count)


app_name
Dota 2                            73541
Rocket League                     54227
The Witcher 3: Wild Hunt          31853
DOOM                              20682
Counter-Strike                    12353
Call of Duty: Modern Warfare 3     5197
NBA 2K16                           2534
The Elder Scrolls V: Skyrim        2260
Football Manager 2016              1742
Name: count, dtype: int64


In [5]:
!pip install transformers torch



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import pandas as pd
from tqdm import tqdm
import time

# Load sentiment model from "siebert/sentiment-roberta-large-english"
model_name = "siebert/sentiment-roberta-large-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Clean the DataFrame: drop rows with NaN or empty 'review_text'
df = df.dropna(subset=['review_text'])
df = df[df['review_text'] != '']

def get_sentiments_batch(text_list, batch_size=32):
    sentiments = []
    total = len(text_list)
    start_time = time.time()

    # Process batches with tqdm for progress tracking
    for i in tqdm(range(0, total, batch_size), desc="Processing Batches", unit="batch"):
        batch = text_list[i:i+batch_size]

        # Debug: log the sample text from this batch (first 100 characters)
        print(f"DEBUG - Processing batch starting at index {i}. Sample text: {batch[0][:100]}...")

        # Tokenize the batch
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(device)

        # Perform inference
        with torch.no_grad():
            outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

        # Extract sentiment and confidence for each sample in the batch
        batch_sentiments = [(torch.argmax(p).item(), torch.max(p).item()) for p in probs]
        sentiments.extend(batch_sentiments)

        # Log sentiment and confidence for the first sample in the batch
        sample_sentiment, sample_confidence = batch_sentiments[0]
        print(f"DEBUG - Batch sample output: Sentiment: {sample_sentiment}, Confidence: {sample_confidence:.2f}")

        # Logging progress
        processed = min(i + batch_size, total)
        elapsed = time.time() - start_time
        batches_done = (i // batch_size) + 1
        avg_time_per_batch = elapsed / batches_done
        batches_remaining = ((total - i) // batch_size)
        estimated_remaining = avg_time_per_batch * batches_remaining

        print(f"Processed {processed}/{total} reviews. Time elapsed: {elapsed:.2f}s. Estimated time remaining: {estimated_remaining:.2f}s")

    return sentiments

# Process reviews in batches
reviews = df['review_text'].tolist()
sentiments = get_sentiments_batch(reviews, batch_size=512)  # Adjust batch_size as needed

# Add sentiment and confidence to the DataFrame
df['sentiment'] = [s[0] for s in sentiments]
df['confidence'] = [s[1] for s in sentiments]

# Output the results to a CSV file
output_csv = "/content/drive/MyDrive/output_sentiments.csv"
df.to_csv(output_csv, index=False)
print(f"Processing complete. Output saved to {output_csv}")


Processing Batches:   0%|          | 0/390 [00:00<?, ?batch/s]

DEBUG - Processing batch starting at index 0. Sample text: ruined my life...


Processing Batches:   0%|          | 1/390 [00:09<1:01:45,  9.52s/batch]

DEBUG - Batch sample output: Sentiment: 0, Confidence: 1.00
Processed 512/199235 reviews. Time elapsed: 9.53s. Estimated time remaining: 3706.11s
DEBUG - Processing batch starting at index 512. Sample text: u dont have nospread lel this game only for legit...


Processing Batches:   1%|          | 2/390 [00:17<54:21,  8.41s/batch]  

...

Processing Batches: 100%|█████████▉| 389/390 [58:59<00:09,  9.23s/batch]

DEBUG - Batch sample output: Sentiment: 0, Confidence: 1.00
Processed 199168/199235 reviews. Time elapsed: 3539.46s. Estimated time remaining: 9.10s
DEBUG - Processing batch starting at index 199168. Sample text: i am unhappy with valves decision to allow mods to be placed behind a paywall...


Processing Batches: 100%|██████████| 390/390 [59:00<00:00,  9.08s/batch]

DEBUG - Batch sample output: Sentiment: 0, Confidence: 1.00
Processed 199235/199235 reviews. Time elapsed: 3540.42s. Estimated time remaining: 0.00s


Processing complete. Output saved to /content/drive/MyDrive/output_sentiments.csv
